In [110]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import urllib
import pyodbc
import os

def connect_perseo_db():
    server = os.environ['PERSEO_IP']
    database = os.environ['PERSEO_DB']
    username = os.environ['PERSEO_USR']
    password = os.environ['PERSEO_PWD']
    
    params = urllib.parse.quote_plus("DRIVER={FreeTDS};"
                                     "SERVER="+server+";"
                                     "PORT=1433;"
                                     "DATABASE="+database+";"
                                     "UID="+username+";"
                                     "PWD="+password+";"
                                     "Trusted_Connection=no")
    
    conn = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
    return conn

def readSqlData(file_name):
    file = open(file_name, mode='r')
    query = file.read()
    file.close()

    engine = connect_perseo_db()
    return pd.read_sql(query, engine)

In [96]:
# Leggi l'elenco attività
dati = readSqlData('Attività/Elenco Servizi.sql')

dati[['OreAula', 'OreStage']] = dati[['OreAula', 'OreStage']].fillna(0)
dati = dati.set_index('IDedizione')

In [97]:
# Aggiungi i dati sulle doti (numero e importo)
doti = readSqlData('Ricavi/Doti.sql')

doti = doti.set_index('IDedizione')

dati = dati.join(doti[['NumeroDoti', 'ImportoDoti']])
dati[['NumeroDoti', 'ImportoDoti']] = dati[['NumeroDoti', 'ImportoDoti']].fillna(0)

In [98]:
# Aggiungi i dati sulle fatture collaboratori
fatture = readSqlData('Costi/Fatture collaboratori.sql')

fatture['Attivita'] = fatture['TipoAttivita'].map({
    'DOCENZA': 'Docenza',
    'ELABORAZIONE MATERIALE DIDATTICO': 'Docenza',
    'FORMAZIONE IN ASSETTO LAVORATIVO EXTRA MONTE ORE': 'Docenza',
    'TUTOR': 'Tutoraggio',
    'CODOCENZA': 'Docenza',
    'SOSTEGNO': 'Tutoraggio',
    'ATTIVITÀ PREVISTA DAL PROGETTO': 'Amministrazione e segreteria',
    'SERVIZI PERSONALIZZATI': 'Tutoraggio',
    'TUTOR D\'AULA': 'Tutoraggio',
    'SOSTEGNO FUORI AULA': 'Tutoraggio',
    'ORIENTAMENTO': 'Orientamento',
    'COORDINAMENTO': 'Progettazione e coordinamento',
    'ESAME': 'Esami',
    'SOSTEGNO FINANZIATO': 'Tutoraggio',
    'PROMOZIONE INTERVENTO': 'Promozione',
    'SOSTEGNO FINANZIATO FUORI AULA': 'Tutoraggio',
    'PROGETTAZIONE E PROGRAMMAZIONE DIDATTICA': 'Progettazione e coordinamento',
    'MONITORAGGIO IN ITINERE ED EX POST': 'Amministrazione e segreteria',
    'VERIFICA/VALUTAZIONE DEGLI APPRENDIMENTI': 'Esami',
    'SERVIZI AL LAVORO': 'Amministrazione e segreteria',
    'ELABORAZIONE REPORTS E STUDI': 'Amministrazione e segreteria',
    'AMMINISTRAZIONE': 'Amministrazione e segreteria',
    'SELEZIONE': 'Amministrazione e segreteria',
    'CODOCENZA ESAME': 'Esami',
    'ATTIVITÀ DI RECUPERO': 'Docenza',
    'COACHING': 'Tutoraggio',
    'DIREZIONE': 'Amministrazione e segreteria',
    'AUSILIARIO': 'Tutoraggio',
    'RICERCA': 'Promozione',
    'SCOUTING': 'Promozione',
    'SEGRETERIA': 'Amministrazione e segreteria',
    'ANALISI DEI BISOGNI': 'Progettazione e coordinamento',
    'FORMAZIONE IN ASSETTO LAVORATIVO': 'Amministrazione e segreteria'
})

fatture = fatture[['IDedizione', 'Attivita', 'sngTotale' ]].groupby(['IDedizione', 'Attivita']).sum()
fatture = fatture.reset_index(level=['IDedizione', 'Attivita'])
fatture = fatture.pivot(index='IDedizione', columns='Attivita', values='sngTotale')
fatture.fillna(0)

dati = dati.join(fatture)

In [99]:
dati

,AnnoAmm,AnnoBando,TipoBando,TipoFormativoInterno,SiglaSede,DescrEdizione,CodiceEdizione,TipoSettoreInt,IDprogetto,CodiceProgetto,...,NIscr,NumeroDoti,ImportoDoti,Amministrazione e segreteria,Docenza,Esami,Orientamento,Progettazione e coordinamento,Promozione,Tutoraggio
IDedizione,,,,,,,,,,,,,,,,,,,,,
26970,2018/2019,2018/2019,BANDO ATTIVITÀ A PAGAMENTO - LIBERO MERCATO,ATTIVITÀ DI RECUPERO DEBITI FORMATIVI,VARESE,SICUREZZA - RECUPERO DDIF 18/19,SICUREZZA - RECUPERO DDIF 18/19,SICUREZZA,3060,LIB 18-19,...,34,0.0,0.0,NaN,174.720001,NaN,NaN,NaN,NaN,NaN
27021,2018/2019,2018/2019,BANDO ATTIVITÀ A PAGAMENTO - LIBERO MERCATO,FORMAZIONE PERMANENTE,BUSTO ARSIZIO 143,FORMAZIONE FGAS IN PREPARAZIONE ESAME PATENTIN...,FGAS,"ELETTRICITÀ, ELETTRONICA, IMPIANTI ELETTRICI",3060,LIB 18-19,...,8,0.0,0.0,NaN,867.519989,NaN,NaN,NaN,NaN,NaN
25917,2018/2019,2018/2019,BANDO ATTIVITÀ A PAGAMENTO - LIBERO MERCATO,FORMAZIONE PERMANENTE,BERGAMO,TECNICHE DI MOVIMENTAZIONE MERCI IN AMBITO AER...,206678,TRASPORTI,3060,LIB 18-19,...,18,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26104,2018/2019,2018/2019,BANDO ATTIVITÀ A PAGAMENTO - LIBERO MERCATO,FORMAZIONE PERMANENTE,CREMONA,SMALTO SEMIPERMANENTE,SMALTO SEMIP.,ESTETICA E BENESSERE,3060,LIB 18-19,...,10,0.0,0.0,NaN,92.000000,NaN,NaN,NaN,NaN,NaN
26853,2018/2019,2018/2019,BANDO ATTIVITÀ A PAGAMENTO - LIBERO MERCATO,FORMAZIONE CONTINUA (CORSI AZIENDALI PER OCCUP...,BERGAMO,AGGIORNAMENTO PER ADDETTI ALLE ATTIVITÀ DI RIM...,AGGIORNAMENTO ADDETTI AMIANTO 4 ED.,EDILIZIA,3060,LIB 18-19,...,15,0.0,0.0,NaN,800.320007,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31122,2021/2022,2021/2022,BANDO CON GARA DI APPALTO AMMINISTRAZIONE PUBB...,FORMAZIONE DETENUTI ED EX-DETENUTI,VARESE,STRUMENTI E TECNICHE DI TINTEGGIATURA,1761 - TINTEGGIATURA,EDILIZIA,3578,CC VARESE,...,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31139,2021/2022,2020/2021,BANDO FONDI INTER-PROFESSIONALI,FORMAZIONE CONTINUA (CORSI AZIENDALI PER OCCUP...,BUSTO ARSIZIO 143,ENAIP LOMBARDIA ED.16 LAVORARE IN UN' OTTICA D...,61952 - VECCHIO CONVENTO,ALBERGHIERO E DELLA RISTORAZIONE,3580,0015711-61952,...,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31138,2021/2022,2020/2021,BANDO FONDI INTER-PROFESSIONALI,FORMAZIONE CONTINUA (CORSI AZIENDALI PER OCCUP...,VIGEVANO,ENAIP LOMBARDIA ED.15 LAVORARE IN UN' OTTICA D...,61783 - INTERAZIENDALE,ESTETICA E BENESSERE,3581,0015711-61783,...,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
# Aggiungi i dati sulle parcelle collaboratori
parcelle = readSqlData('Costi/Parcelle collaboratori.sql')
parcelle = parcelle[parcelle['IDedizione'].notna()]

parcelle['Attivita'] = parcelle['TipoCausaleParcella'].map({
    'PRATICHE NOTARILI PER ATS': 'Amministrazione e segreteria',
    'ALTRO': 'Amministrazione e segreteria',
    'COMMISSIONE DI ESAME': 'Esami',
    'ADEMPIMENTI L.626': 'Amministrazione e segreteria',
    'CONSULENZA': 'Amministrazione e segreteria'
})

parcelle = parcelle[['IDedizione', 'Attivita', 'sngTotale' ]].groupby(['IDedizione', 'Attivita']).sum()
parcelle = parcelle.reset_index(level=['IDedizione', 'Attivita'])

parcelle = parcelle.pivot(index='IDedizione', columns='Attivita', values='sngTotale')
parcelle.fillna(0)
parcelle = parcelle.reset_index()
parcelle

dati.merge(parcelle, how='left', on='IDedizione').set_index()
#pd.merge(dati, parcelle, on=parcelle.columns)#.set_index(cols).sum(axis=1)
#dati = dati.join(parcelle)
#dati['ParcelleCollaboratori'] = dati['ParcelleCollaboratori'].fillna(0)

Attivita,IDedizione,Amministrazione e segreteria,Esami
0,16076.0,1768.000000,NaN
1,25887.0,2105.129883,NaN
2,25905.0,622.200012,NaN
3,26286.0,NaN,840.0
4,26287.0,NaN,550.0
5,26808.0,NaN,368.0
6,27118.0,3150.000000,NaN
7,27478.0,NaN,1056.0
8,27989.0,1012.000000,NaN
9,28098.0,640.000000,NaN


In [75]:
# Salva i dati in excel
dati = dati.reset_index(level=['Attivita'])
dati.to_excel("output.xlsx", sheet_name='data')

KeyError: 'Requested level (Attivita) does not match index name (IDedizione)'